In [2]:
import numpy             as np
import matplotlib.pyplot as plt
from matplotlib          import rc 
from matplotlib          import cm
from mpl_toolkits.mplot3d import Axes3D

import scipy.integrate as integrate
import os.path as path
import wget

# Maximum Likelihood estimation